Demonstrate the use of Python for image processing: contrast stretching and linear filtering and plot its histogram.
Exercise 1: Contrast Stretching, Linear Filtering, and Histogram Plotting
In this task:

Perform contrast stretching on an image.
Apply a linear filter (such as a Gaussian filter).
Plot the histogram of the original and processed images.

In [ ]:
!pip install opencv-python
!pip install numpy
!pip install matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load an image in grayscale
image = cv2.imread('new.jpg', cv2.IMREAD_GRAYSCALE)

# 1. Enhanced Contrast Stretching
min_bound, max_bound = 50, 200  # Set custom lower and upper bounds
contrast_stretched = np.clip(((image - min_bound) / (max_bound - min_bound)) * 255, 0, 255).astype(np.uint8)

# 2. Stronger Gaussian Blur
gaussian_blur = cv2.GaussianBlur(contrast_stretched, (31, 31), 0)  # Increased kernel size to (15, 15)

# 3. Plotting the Histograms
plt.figure(figsize=(12, 8))

# Original Image and its Histogram
plt.subplot(2, 3, 1)
plt.imshow(image, cmap='gray')
plt.title("Original Image")
plt.axis('off')

plt.subplot(2, 3, 4)
plt.hist(image.ravel(), bins=256, range=(0, 256))
plt.title("Histogram - Original")

# Contrast Stretched Image and Histogram
plt.subplot(2, 3, 2)
plt.imshow(contrast_stretched, cmap='gray')
plt.title("Contrast Stretched Image")
plt.axis('off')

plt.subplot(2, 3, 5)
plt.hist(contrast_stretched.ravel(), bins=256, range=(0, 256))
plt.title("Histogram - Contrast Stretched")

# Gaussian Blurred Image and Histogram
plt.subplot(2, 3, 3)
plt.imshow(gaussian_blur, cmap='gray')
plt.title("Gaussian Blurred Image")
plt.axis('off')

plt.subplot(2, 3, 6)
plt.hist(gaussian_blur.ravel(), bins=256, range=(0, 256))
plt.title("Histogram - Gaussian Blurred")

plt.tight_layout()
plt.show()


In [ ]:
!pip install tensorflow matplotlib numpy

Classify the images using Horse and Human dataset, and do the following using binary classification:
a.	Load the dataset
b.	View the no. of testing and training images
c.	plot some images
d.	Normalizing the training data
e.	Build a convolutional neural network (ResNet) to train and testing the images
f.	Show the training and testing accuracy

In [3]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
train_dir = 'dataset'  
datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)  # 20% of data as validation

train_data = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128), 
    batch_size=32,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


In [ ]:
print(f"Number of training images: {train_data.samples}")
print(f"Number of validation images: {val_data.samples}")


In [ ]:
def plot_images(images, labels):
    plt.figure(figsize=(10, 10))
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        plt.title("Horse" if labels[i] == 0 else "Human")
        plt.axis('off')
    plt.show()

images, labels = next(train_data)
plot_images(images, labels)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  

model = Model(inputs=base_model.input, outputs=x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=10  
)


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.show()
